In [42]:
import os 
import dart_fss as dart 

# 메타 참고 문서
- https://purplechip.tistory.com/32

# KRX 메타 데이터 수집 

In [43]:
dart.set_api_key(api_key=os.environ['DART_PSS_API_KEY'])

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

print(f'공시된 기업 전체 수: {len(corp_list)}')

공시된 기업 전체 수: 101454


In [44]:
type(corp_list)

dart_fss.corp.corp_list.CorpList

### corp_cls
> ‘Y’: 코스피, ‘K’: 코스닥, ‘N’: 코넥스, ‘E’: 기타

In [45]:
# 코스피 기업정보들
stock_corps = [] 

for corp in corp_list:
    try:
        if corp.corp_cls == "Y" or corp.corp_cls == "K": # 코스피/코스탁 기업정보 KOSPI
            kospi_corp = {
                "corp_code": corp.corp_code if corp.corp_code else None,
                "corp_name": corp.corp_name if corp.corp_name else None,
                "stock_code": corp.stock_code if corp.stock_code else None, 
                "corp_cls": corp.corp_cls if corp.corp_cls else None,
                "corp_cls_nm": "kospi" if corp.corp_cls == "Y" else "kosdaq",
                "sector": corp.sector if corp.sector else None,
                "product": corp.product if corp.product else None
            }

            stock_corps.append(kospi_corp)

    except:
        pass

In [46]:
len(stock_corps)

2481

In [47]:
stock_corps[0]

{'corp_code': '00956028',
 'corp_name': '엑세스바이오',
 'stock_code': '950130',
 'corp_cls': 'K',
 'corp_cls_nm': 'kosdaq',
 'sector': '의료용품 및 기타 의약 관련제품 제조업',
 'product': '말라리아 진단키트(RDT), HIV 진단키트(RDT)'}

In [48]:
import pandas as pd 

In [49]:
df_krx = pd.DataFrame(stock_corps)

print(df_krx.shape)
df_krx.head()

(2481, 7)


,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,sector,product
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름"
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


In [50]:
df_krx.rename(columns={'sector': 'industry_krx_nm', 'product': 'industry_krx_desc'}, inplace=True)

df_krx.head()

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
0,00956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,00783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,01170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름"
3,00600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,None
4,01139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


In [51]:
df_krx[df_krx['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
1761,00126380,삼성전자,005930,Y,kospi,통신 및 방송 장비 제조업,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조..."


In [52]:
save_path_krx = '../data/meta/stock_corps_krx.csv'
df_krx.to_csv(save_path_krx, header=True, index=False, encoding='utf-8')

## 데이터 조회

In [53]:
import pandas as pd 

In [54]:
df_krx = pd.read_csv(save_path_krx)

df_krx.head()

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
0,956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,1170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름"
3,600013,맵스리얼티1,94800,Y,kospi,신탁업 및 집합투자업,NaN
4,1139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


# [WICS 메타 크롤링](https://www.wiseindex.com/About/WICS)

In [18]:
wics_lc = {10:'에너지', 
           15:'소재', 
           20:'산업재', 
           25:'경기관련소비재', 
           30:'필수소비재', 
           35:'건강관리',
           40:'금융', 
           45:'IT', 
           50:'커뮤니케이션서비스', 
           55:'유틸리티'}
 
wics_mc = {1010:'에너지',
           1510:'소재',
           2010:'자본재',
           2020:'상업서비스와공급품',
           2030:'운송',
           2510:'자동차와부품',
           2520:'내구소비재와의류',
           2530:'호텔,레스토랑,레저 등',
           2550:'소매(유통)',
           2560:'교육서비스',
           3010:'식품과기본식료품소매',
           3020:'식품,음료,담배',
           3030:'가정용품과개인용품',
           3510:'건강관리장비와서비스',
           3520:'제약과생물공학',
           4010:'은행',
           4020:'증권',
           4030:'다각화된금융',
           4040:'보험',
           4050:'부동산',
           4510:'소프트웨어와서비스',
           4520:'기술하드웨어와장비',
           4530:'반도체와반도체장비',
           4535:'전자와 전기제품',
           4540:'디스플레이',
           5010:'전기통신서비스',
           5020:'미디어와엔터테인먼트',
           5510:'유틸리티'}
 
def wics_url(date, wics_code):
    '''
    Parameter
    - date[str] : the date corresponding data (yyyymmdd)
    - wics_code[int] : the wics code corresponding data (use wics_lc or wics_mc)
    
    Return
    - url[str]
    '''
    url ='http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&'\
          'dt=' + date + '&sec_cd=G' + str(wics_code)
    return url
 
def comp_url(code):
    '''
    Parameter
    - code[str] : the company code corresponding data
    
    Return
    - url[str]
    '''
    url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&'\
          'gicode=A' + code + \
          '&cID=&MenuYn=Y&ReportGB=&NewMenuID=Y&stkGb=701'
    return url


In [19]:
import requests
from datetime import datetime 
from pandas import DataFrame


In [20]:
today = datetime.now().strftime("%Y%m%d")
today 

'20230704'

In [31]:
today = '20230703'

In [32]:
result_list = []

In [33]:
for wics_code in wics_mc.keys():
    response = requests.get(wics_url(today, wics_code))
    # print(response.status_code)
    wics_code = str(wics_code)
    if(response.status_code == 200): # request success
            json_list = response.json() # dictionary
            # response.text -> return str type
            for json in json_list['list']:
                ls = json['SEC_NM_KOR'] # Large sector
                ms = json['IDX_NM_KOR'][5:] # Medium sector
                code = json['CMP_CD'] # Company code
                name = json['CMP_KOR'] # Company korean name
                result_list.append({'stock_code':code, 'corp_name':name, 'industry_wics_ls_nm':ls, 'industry_wics_ls_cd':wics_code[:2], 'industry_wics_ms_nm':ms, 'industry_wics_ms_cd':wics_code})
    else:
        print('Error:' + response.status_code)
        print('WICS code:' + str(wics_code))

In [34]:
df_wics = DataFrame(result_list)

print(df_wics.shape)
df_wics.head()

(2390, 6)


,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd
0,096770,SK이노베이션,에너지,10,에너지,1010
1,009830,한화솔루션,에너지,10,에너지,1010
2,010950,S-Oil,에너지,10,에너지,1010
3,267250,HD현대,에너지,10,에너지,1010
4,112610,씨에스윈드,에너지,10,에너지,1010


In [35]:
df_wics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390 entries, 0 to 2389
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   stock_code           2390 non-null   object
 1   corp_name            2390 non-null   object
 2   industry_wics_ls_nm  2390 non-null   object
 3   industry_wics_ls_cd  2390 non-null   object
 4   industry_wics_ms_nm  2390 non-null   object
 5   industry_wics_ms_cd  2390 non-null   object
dtypes: object(6)
memory usage: 112.2+ KB


In [72]:
save_path_wics = '../data/meta/stock_corps_wics.csv'

df_wics.to_csv(save_path_wics, header=True, index=False, encoding='utf-8')

In [75]:
df_wics[df_wics['corp_name'] == '삼성전자']

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd
1970,005930,삼성전자,IT,45,반도체와반도체장비,4530


In [110]:
df_wics[df_wics['corp_name'] == '카카오']

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd
2256,035720,카카오,커뮤니케이션서비스,50,미디어와엔터테인먼트,5020


In [64]:
df_krx[df_krx['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
1761,126380,삼성전자,5930,Y,kospi,통신 및 방송 장비 제조업,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조..."


## WICS 소분류 크롤링

In [11]:
import requests
import pandas as pd 

from bs4 import BeautifulSoup as bs

In [4]:
# url = 'https://finance.naver.com/item/coinfo.naver?code=005930'
url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930'
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Referer':'https://finance.naver.com/item/coinfo.naver?code=005930'
}
response = requests.get(url, headers=headers)

In [5]:
response.status_code

200

In [6]:
response.text

'\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html lang="ko">\r\n<head>\r\n\t<title>온라인기업정보 - 기업모니터 - 기업개요(삼성전자)</title>\r\n\t<link rel="shortcut icon" href="/favicon.ico" />\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<script language="javascript" src="/include/domain.js" type="text/javascript"></script>\r\n\t<!--[if (!IE) | (gt IE 8)]>\r\n    \t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />\r\n    <![endif]-->\r\n\t<!--[if IE 6]>\r\n            <link href="/include/only_ie6.css" type="text/css" rel="stylesheet"/>\r\n        <![endif]-->\r\n\t<!--[if lt IE 8]>\r\n            <link href="/include/lt_ie8.css" type="text/css" rel="stylesheet"/>\r\n        <![endif]-->\r\n\t<link href="/v2/include/css?v=q7Pq73b9jj00gmFmOVUSxzXv_5FtLNW9qxkqFwJqh-g1" rel="stylesheet"/>\r\n\r\n\t<link href="/v2/include/css1?v=2Kc54WAY0-YKwI0WYHBphRe4FmbyttZ0_0A_hA

In [7]:
soup = bs(response.text, "html.parser")

In [8]:
soup.select(".line-left") #[0].get_text()

[<dt class="line-left"><a href="../company/c1020001.aspx?cmp_cd=005930&amp;cn=">기업개요</a></dt>,
 <dt class="line-left"><a href="../company/c1030001.aspx?cmp_cd=005930&amp;cn=">재무분석</a></dt>,
 <dt class="line-left"><a href="../company/c1040001.aspx?cmp_cd=005930&amp;cn=">투자지표</a></dt>,
 <dt class="line-left"><a href="../company/c1050001.aspx?cmp_cd=005930&amp;cn=">컨센서스</a></dt>,
 <dt class="line-left"><a href="../company/c1060001.aspx?cmp_cd=005930&amp;cn=">업종분석</a></dt>,
 <dt class="line-left"><a href="../company/c1090001.aspx?cmp_cd=005930&amp;cn=">섹터분석</a></dt>,
 <dt class="line-left"><a href="../company/c1070001.aspx?cmp_cd=005930&amp;cn=">지분현황</a></dt>,
 <dt class="line-left">SamsungElec</dt>,
 <dt class="line-left">KOSPI : 전기전자</dt>,
 <dt class="line-left">WICS : 반도체와반도체장비</dt>,
 <dt class="line-left">BPS <b class="num">51,529</b></dt>,
 <dt class="line-left">PER <b class="num">11.02</b></dt>,
 <dt class="line-left">업종PER <b class="num">12.89</b></dt>,
 <dt class="line-left">PBR <b

In [9]:
for node in soup.select(".line-left"):
    if 'WICS' in node.get_text():
        print(node.get_text())
        break 

WICS : 반도체와반도체장비


In [38]:
save_path_wics = '../data/meta/stock_corps_wics.csv'
df_dtype = {
    'stock_code': 'object'
}

df_wics = pd.read_csv(save_path_wics, dtype=df_dtype)
df_wics.head()

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd
0,096770,SK이노베이션,에너지,10,에너지,1010
1,009830,한화솔루션,에너지,10,에너지,1010
2,010950,S-Oil,에너지,10,에너지,1010
3,267250,HD현대,에너지,10,에너지,1010
4,112610,씨에스윈드,에너지,10,에너지,1010


In [45]:
def add_industry_wics_ss_nm(stock_code):
    url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={stock_code}'.format(stock_code=stock_code)
    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Referer':'https://finance.naver.com/item/coinfo.naver?code={stock_code}'.format(stock_code=stock_code)
    }
    response = requests.get(url, headers=headers)

    soup = bs(response.text, "html.parser")
    for node in soup.select(".line-left"):
        if 'WICS' in node.get_text():
            return node.get_text().replace('WICS : ', '')

In [47]:
df_wics['industry_wics_ss_nm'] = df_wics['stock_code'].map(add_industry_wics_ss_nm)

df_wics.head()

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm
0,096770,SK이노베이션,에너지,10,에너지,1010,석유와가스
1,009830,한화솔루션,에너지,10,에너지,1010,에너지장비및서비스
2,010950,S-Oil,에너지,10,에너지,1010,석유와가스
3,267250,HD현대,에너지,10,에너지,1010,석유와가스
4,112610,씨에스윈드,에너지,10,에너지,1010,에너지장비및서비스


In [50]:
df_wics[df_wics['corp_name'] == '삼성전자']

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm
1970,005930,삼성전자,IT,45,반도체와반도체장비,4530,반도체와반도체장비


In [51]:
df_wics[df_wics['corp_name'] == '카카오']

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm
2256,035720,카카오,커뮤니케이션서비스,50,미디어와엔터테인먼트,5020,양방향미디어와서비스


In [48]:
df_wics.isnull().sum()

stock_code             0
corp_name              0
industry_wics_ls_nm    0
industry_wics_ls_cd    0
industry_wics_ms_nm    0
industry_wics_ms_cd    0
industry_wics_ss_nm    0
dtype: int64

- 코드 추가 

In [60]:
wics_ss_dic = {
    # 에너지
    '에너지장비및서비스': 101010,
    '석유와가스': 101020,
    # 소재
    '화학': 151010,
    '포장재': 151030,
    '비철금속': 151040,
    '철강': 151050,
    '종이와목재': 151060,
    # 자본재
    '우주항공과국방': 201010,
    '건축제품': 201020,
    '건축자재': 201025,
    '건설': 201030,
    '가구': 201035,
    '전기장비': 201040,
    '복합기업': 201050,
    '기계': 201060,
    '조선': 201065,
    '무역회사와판매업체': 201070,
    # 상업서비스와공급품
    '상업서비스와공급품': 202010,
    # 운송
    '항공화물운송과물류': 203010,
    '항공사': 203020,
    '해운사': 203030,
    '도로와철도운송': 203040,
    '운송인프라': 203050,
    # 자동차와부품
    '자동차부품': 251010,
    '자동차': 251020,
    # 내구소비재와의류
    '가정용기기와용품': 252040,
    '레저용장비와제품': 252050,
    '섬유,의류,신발,호화품': 252060,
    '화장품': 252065,
    '문구류': 252070,
    # 호텔,레스토랑,레저 등
    '호텔,레스토랑,레저': 253010,
    '다각화된소비자서비스': 253020,
    # 소매(유통)
    '판매업체': 255010,
    '인터넷과카탈로그소매': 255020,
    '백화점과일반상점': 255030,
    '전문소매': 255040,
    # 교육서비스
    '교육서비스': 256010,
    # 식품과기본식료품소매
    '식품과기본식료품소매': 301010,
    # 식품,음료,담배
    '음료': 302010,
    '식품': 302020,
    '담배': 302030,
    # 가정용품과개인용품
    '가정용품': 303010,
    # 건강관리장비와서비스
    '건강관리장비와용품': 351010,
    '건강관리업체및서비스': 351020,
    '건강관리기술': 351030,
    # 제약과생물공학
    '생물공학': 352010,
    '제약': 352020,
    '생명과학도구및서비스': 352030,
    # 은행
    '은행': 401010,
    # 증권
    '증권': 402010,
    # 다각화된금융
    '창업투자': 403020,
    '카드': 403030,
    '기타금융': 403040,
    # 보험
    '손해보험': 404010,
    '생명보험': 404020,
    # 부동산
    '부동산': 405020,
    # 소프트웨어와서비스
    'IT서비스': 451020,
    '소프트웨어': 451030,
    # 기술하드웨어와장비
    '통신장비': 452010,
    '핸드셋': 452015,
    '컴퓨터와주변기기': 452020,
    '전자장비와기기': 452030,
    '사무용전자제품': 452040,
    # 반도체와반도체장비
    '반도체와반도체장비': 453010,
    # 전자와 전기제품
    '전자제품': 453510,
    '전기제품': 453520,
    # 디스플레이
    '디스플레이패널': 454010,
    '디스플레이장비및부품': 454020,
    # 전기통신서비스
    '다각화된통신서비스': 501010,
    '무선통신서비스': 501020,
    # 미디어와엔터테인먼트
    '광고': 502010,
    '방송과엔터테인먼트': 502020,
    '출판': 502030,
    '게임엔터테인먼트': 502040,
    '양방향미디어와서비스': 502050,
    # 유틸리티
    '전기유틸리티': 551010,
    '가스유틸리티': 551020,
    '복합유틸리티': 551030,
    '독립전력생산및에너지거래': 551050
}

In [61]:
df_wics['industry_wics_ss_cd'] = df_wics['industry_wics_ss_nm'].map(wics_ss_dic)

In [62]:
df_wics.head()

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,096770,SK이노베이션,에너지,10,에너지,1010,석유와가스,101020
1,009830,한화솔루션,에너지,10,에너지,1010,에너지장비및서비스,101010
2,010950,S-Oil,에너지,10,에너지,1010,석유와가스,101020
3,267250,HD현대,에너지,10,에너지,1010,석유와가스,101020
4,112610,씨에스윈드,에너지,10,에너지,1010,에너지장비및서비스,101010


In [64]:
df_wics.isnull().sum()

stock_code             0
corp_name              0
industry_wics_ls_nm    0
industry_wics_ls_cd    0
industry_wics_ms_nm    0
industry_wics_ms_cd    0
industry_wics_ss_nm    0
industry_wics_ss_cd    0
dtype: int64

In [65]:
save_path_wics = '../data/meta/stock_corps_wics.csv'

df_wics.to_csv(save_path_wics, header=True, index=False, encoding='utf-8')

In [66]:
save_path_wics = '../data/meta/stock_corps_wics.csv'
df_dtype = {
    'stock_code': 'object'
}

df_wics = pd.read_csv(save_path_wics, dtype=df_dtype)

print(f'{df_wics.shape} / {df_wics.isnull().sum().sum()}')
df_wics.head()

(2390, 8) / 0


,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,096770,SK이노베이션,에너지,10,에너지,1010,석유와가스,101020
1,009830,한화솔루션,에너지,10,에너지,1010,에너지장비및서비스,101010
2,010950,S-Oil,에너지,10,에너지,1010,석유와가스,101020
3,267250,HD현대,에너지,10,에너지,1010,석유와가스,101020
4,112610,씨에스윈드,에너지,10,에너지,1010,에너지장비및서비스,101010


# Create stock_corps.csv

In [94]:
save_path_wics = '../data/meta/stock_corps_wics.csv'
df_dtype = {
    'stock_code': 'object'
}

df_wics = pd.read_csv(save_path_wics, dtype=df_dtype)

print(f'{df_wics.shape} / {df_wics.isnull().sum().sum()}')
df_wics.head()

(2390, 8) / 0


,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,096770,SK이노베이션,에너지,10,에너지,1010,석유와가스,101020
1,009830,한화솔루션,에너지,10,에너지,1010,에너지장비및서비스,101010
2,010950,S-Oil,에너지,10,에너지,1010,석유와가스,101020
3,267250,HD현대,에너지,10,에너지,1010,석유와가스,101020
4,112610,씨에스윈드,에너지,10,에너지,1010,에너지장비및서비스,101010


In [95]:
df_wics[df_wics['corp_name'] == '삼성전자']

,stock_code,corp_name,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
1970,005930,삼성전자,IT,45,반도체와반도체장비,4530,반도체와반도체장비,453010


In [96]:
save_path_krx = '../data/meta/stock_corps_krx.csv'
df_dtype = {
    'stock_code': 'object'
}

df_krx = pd.read_csv(save_path_krx, dtype=df_dtype)

print(f'{df_krx.shape} / {df_krx.isnull().sum().sum()}')
df_krx.head()

(2481, 7) / 14


,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
0,956028,엑세스바이오,950130,K,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,783246,글로벌에스엠,900070,K,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,1170962,GRT,900290,K,kosdaq,기타 금융업,"광학필름, 포장필름"
3,600013,맵스리얼티1,094800,Y,kospi,신탁업 및 집합투자업,NaN
4,1139266,로스웰,900260,K,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


In [97]:
df_krx[df_krx['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,corp_cls,corp_cls_nm,industry_krx_nm,industry_krx_desc
1761,126380,삼성전자,005930,Y,kospi,통신 및 방송 장비 제조업,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조..."


## merge

In [98]:
df_krx = df_krx[['stock_code', 'corp_name', 'corp_cls_nm', 'industry_krx_nm', 'industry_krx_desc']]
df_krx.head()

,stock_code,corp_name,corp_cls_nm,industry_krx_nm,industry_krx_desc
0,950130,엑세스바이오,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)"
1,900070,글로벌에스엠,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)"
2,900290,GRT,kosdaq,기타 금융업,"광학필름, 포장필름"
3,094800,맵스리얼티1,kospi,신탁업 및 집합투자업,NaN
4,900260,로스웰,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈"


In [104]:
df_wics = df_wics[['stock_code', 'industry_wics_ls_nm', 'industry_wics_ls_cd', 'industry_wics_ms_nm', 'industry_wics_ms_cd', 'industry_wics_ss_nm', 'industry_wics_ss_cd']]
df_wics.head()

,stock_code,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,096770,에너지,10,에너지,1010,석유와가스,101020
1,009830,에너지,10,에너지,1010,에너지장비및서비스,101010
2,010950,에너지,10,에너지,1010,석유와가스,101020
3,267250,에너지,10,에너지,1010,석유와가스,101020
4,112610,에너지,10,에너지,1010,에너지장비및서비스,101010


In [105]:
df_merge = pd.merge(df_krx, df_wics, how='left', on=['stock_code'])

print(f'{df_merge.shape} / {df_merge.isnull().sum().sum()}')
df_merge.head()

(2481, 11) / 560


,stock_code,corp_name,corp_cls_nm,industry_krx_nm,industry_krx_desc,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,950130,엑세스바이오,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",NaN,NaN,NaN,NaN,NaN,NaN
1,900070,글로벌에스엠,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",NaN,NaN,NaN,NaN,NaN,NaN
2,900290,GRT,kosdaq,기타 금융업,"광학필름, 포장필름",NaN,NaN,NaN,NaN,NaN,NaN
3,094800,맵스리얼티1,kospi,신탁업 및 집합투자업,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,900260,로스웰,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
df_merge.isnull().sum()

stock_code              0
corp_name               0
corp_cls_nm             0
industry_krx_nm         0
industry_krx_desc      14
industry_wics_ls_nm    91
industry_wics_ls_cd    91
industry_wics_ms_nm    91
industry_wics_ms_cd    91
industry_wics_ss_nm    91
industry_wics_ss_cd    91
dtype: int64

In [108]:
c = df_merge['industry_wics_ls_nm'].isnull()

df_merge.loc[c]

,stock_code,corp_name,corp_cls_nm,industry_krx_nm,industry_krx_desc,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,950130,엑세스바이오,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",NaN,NaN,NaN,NaN,NaN,NaN
1,900070,글로벌에스엠,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",NaN,NaN,NaN,NaN,NaN,NaN
2,900290,GRT,kosdaq,기타 금융업,"광학필름, 포장필름",NaN,NaN,NaN,NaN,NaN,NaN
3,094800,맵스리얼티1,kospi,신탁업 및 집합투자업,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,900260,로스웰,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2345,263540,어스앤에어로스페이스,kosdaq,"항공기,우주선 및 부품 제조업",도어시스템,NaN,NaN,NaN,NaN,NaN,NaN
2349,078130,국일제지,kosdaq,기타 종이 및 판지 제품 제조업,"산업용 및 가정용 지류(강판간지, 다공지 등)",NaN,NaN,NaN,NaN,NaN,NaN
2464,068940,셀피글로벌,kosdaq,전자부품 제조업,신용카드 및 전자화폐 등,NaN,NaN,NaN,NaN,NaN,NaN
2465,091090,세원이앤씨,kospi,"구조용 금속제품, 탱크 및 증기발생기 제조업","화학플랜트 설비,기계 제작(음식료,석유화학사업관련)/유공압기기 판매",NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_krx['corp_cls_nm'].value_counts()

corp_cls_nm
kosdaq    1647
kospi      834
Name: count, dtype: int64

In [110]:
df_merge['corp_cls_nm'].value_counts()

corp_cls_nm
kosdaq    1647
kospi      834
Name: count, dtype: int64

In [111]:
df_wics.shape

(2390, 7)

In [112]:
c = df_merge['industry_wics_ls_nm'].isnull()

df_merge.loc[~c].shape 

(2390, 11)

In [113]:
save_path = '../data/meta/stock_corps.csv'

df_merge.to_csv(save_path, header=True, index=False, encoding='utf-8')

In [1]:
import pandas as pd 

In [2]:
save_path = '../data/meta/stock_corps.csv'
df_dtype = {
    'stock_code': 'object'
}

df_corps = pd.read_csv(save_path, dtype=df_dtype)

print(f'{df_corps.shape} / {df_corps.isnull().sum().sum()}')
df_corps.head()

(2481, 11) / 560


,stock_code,corp_name,corp_cls_nm,industry_krx_nm,industry_krx_desc,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
0,950130,엑세스바이오,kosdaq,의료용품 및 기타 의약 관련제품 제조업,"말라리아 진단키트(RDT), HIV 진단키트(RDT)",NaN,NaN,NaN,NaN,NaN,NaN
1,900070,글로벌에스엠,kosdaq,기타 금융업,"자회사 제품 : IT기기용 정밀 화스너부품(스크류, 샤프트, 스프링)",NaN,NaN,NaN,NaN,NaN,NaN
2,900290,GRT,kosdaq,기타 금융업,"광학필름, 포장필름",NaN,NaN,NaN,NaN,NaN,NaN
3,094800,맵스리얼티1,kospi,신탁업 및 집합투자업,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,900260,로스웰,kosdaq,기타 금융업,"CAN Control Gateway, CAN기능모듈, BCM 차체제어모듈",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_corps[df_corps['corp_name'] == '삼성바이오로직스']

,stock_code,corp_name,corp_cls_nm,industry_krx_nm,industry_krx_desc,industry_wics_ls_nm,industry_wics_ls_cd,industry_wics_ms_nm,industry_wics_ms_cd,industry_wics_ss_nm,industry_wics_ss_cd
1618,207940,삼성바이오로직스,kospi,기초 의약물질 및 생물학적 제제 제조업,바이오의약품,건강관리,35.0,제약과생물공학,3520.0,제약,352020.0
